Flight Route 

Leveraging OpenAI gpt-3.5-turbo to find the most optimal flight route from the dataseet.

## Setup

In [ ]:
# make sure to use the latest version of the openai python package
!pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00


In [ ]:
from google.colab import files, drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import openai
import os
import pandas as pd
from pprint import pprint

client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "api_key"))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Since, Fine-tuning works best when focused on a particular domain, the dataset is both focused enough for the model to learn, but general enough that unseen examples won't be missed. 

In [ ]:

airline_df = pd.read_csv("/content/drive/MyDrive/AI Project/final_dataset.csv")

airline_df.head()

,Start Date,Departure Airport,Departure City,Departure Country,Destination Airport,Destination City,Destination Country,Layover Airport,Layover City,Layover Country,Layover Time,Flight Duration,Airline,Flight Number,NER
0,10/08/2023,Ie Shima Auxiliary Air Base,Iejima,Japan,Sandefjord Airport,Torp,Sandefjord,Roudnice Airport,Roudnice nad Lebem,Czech Republic,4.0,6.0,Alexander-Byrd,EAU460,Ie Shima Auxiliary Air Base\nIejima\nJapan\nSa...
1,08/20/2023,Mayor Buenaventura Vivas International Airport,Santo Domingo,Venezuela,Akhiok Airport,Akhiok,United States,CapitÃ¡n Manuel RÃ­os Airbase,Carrizal,Venezuela,5.0,1.0,Cole and Sons,EIZ862,Mayor Buenaventura Vivas International Airport...
2,11/04/2023,Berbera Airport,Berbera,Somalia,Majors Airport,Greenvile,United States,Roxas Airport,Roxas City,Philippines,4.0,12.0,Morgan Inc,RXD676,Berbera Airport\nBerbera\nSomalia\nMajors Airp...
3,09/01/2023,Roshchino International Airport,Tyumen,Russia,Aalborg Airport,Aalborg,Denmark,Benin Airport,Benin,Nigeria,6.0,11.0,Lucero-Lewis,HGE223,Roshchino International Airport\nTyumen\nRussi...
4,05/27/2023,Brustem Airfield Sint Truiden,Sint-truiden,Belgium,Kenneth Kaunda International Airport Lusaka,Lusaka,Zambia,Cozumel International Airport,Cozumel,Mexico,4.0,1.0,Bernard LLC,JRO437,Brustem Airfield Sint Truiden\nSint-truiden\nB...


In [ ]:
training_data = []
system_message = "You are a helpful travel assistant. Given the Start Date, Departure City, Destination City, you will provide the shortest travel details, including departure airport, destination airport, airline, flight number, layover details (if any), and total travel duration."

def create_user_message(row):
    message = f"Start Date: {row['Start Date']}\n"
    message += f"Departure City: {row['Departure City']}\n"
    message += f"Destination City: {row['Destination City']}\n\n"
    message += "Based on the given information, please provide the most suitable travel plan."
    return message

def prepare_example_conversation(row):
    messages = []
    messages.append({"role": "system", "content": system_message})
    user_message = create_user_message(row)
    messages.append({"role": "user", "content": user_message})
    messages.append({"role": "assistant", "content": row["NER"]})
    return {"messages": messages}

# Iterate over your dataset and create the training examples
for index, row in airline_df.iterrows():
    training_data.append(prepare_example_conversation(row))

In [ ]:
# use the first 100 rows of the dataset for training
training_df = airline_df.loc[0:100]

# apply the prepare_example_conversation function to each row of the training_df
training_data = training_df.apply(prepare_example_conversation, axis=1).tolist()

for example in training_data[:5]:
    print(example)

{'messages': [{'role': 'system', 'content': 'You are a helpful travel assistant. Given the Start Date, Departure City, Destination City, you will provide the shortest travel details, including departure airport, destination airport, airline, flight number, layover details (if any), and total travel duration.'}, {'role': 'user', 'content': 'Start Date: 10/08/2023\nDeparture City: Iejima\nDestination City: Torp\n\nBased on the given information, please provide the most suitable travel plan.'}, {'role': 'assistant', 'content': 'Ie Shima Auxiliary Air Base\nIejima\nJapan\nSandefjord Airport\nTorp\nSandefjord\nAlexander-Byrd\nEAU460\nRoudnice Airport\nRoudnice nad Lebem\nCzech Republic\n4.0\n6.0'}]}
{'messages': [{'role': 'system', 'content': 'You are a helpful travel assistant. Given the Start Date, Departure City, Destination City, you will provide the shortest travel details, including departure airport, destination airport, airline, flight number, layover details (if any), and total tra

In [ ]:
validation_df = airline_df.loc[101:200]
validation_data = validation_df.apply(prepare_example_conversation, axis=1).tolist()

In [ ]:
def write_jsonl(data_list: list, filename: str) -> None:
    with open(filename, "w") as out:
        for ddict in data_list:
            jout = json.dumps(ddict) + "\n"
            out.write(jout)

In [ ]:
training_file_name = "airline_details_finetune_training.jsonl"
write_jsonl(training_data, training_file_name)

validation_file_name = "airline_details_finetune_validation.jsonl"
write_jsonl(validation_data, validation_file_name)

This is what the first 5 lines of our training `.jsonl` file look like:

In [ ]:
# print the first 5 lines of the training file
!head -n 5 airline_details_finetune_training.jsonl

{"messages": [{"role": "system", "content": "You are a helpful travel assistant. Given the Start Date, Departure City, Destination City, you will provide the shortest travel details, including departure airport, destination airport, airline, flight number, layover details (if any), and total travel duration."}, {"role": "user", "content": "Start Date: 10/08/2023\nDeparture City: Iejima\nDestination City: Torp\n\nBased on the given information, please provide the most suitable travel plan."}, {"role": "assistant", "content": "Ie Shima Auxiliary Air Base\nIejima\nJapan\nSandefjord Airport\nTorp\nSandefjord\nAlexander-Byrd\nEAU460\nRoudnice Airport\nRoudnice nad Lebem\nCzech Republic\n4.0\n6.0"}]}
{"messages": [{"role": "system", "content": "You are a helpful travel assistant. Given the Start Date, Departure City, Destination City, you will provide the shortest travel details, including departure airport, destination airport, airline, flight number, layover details (if any), and total tra

### Upload files

You can now upload the files to our `Files` endpoint to be used by the fine-tuned model.


In [ ]:
with open(training_file_name, "rb") as training_fd:
    training_response = client.files.create(
        file=training_fd, purpose="fine-tune"
    )

training_file_id = training_response.id

with open(validation_file_name, "rb") as validation_fd:
    validation_response = client.files.create(
        file=validation_fd, purpose="fine-tune"
    )
validation_file_id = validation_response.id

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-IISv5YuyaIun14xPRgHbBYj8
Validation file ID: file-29cO9onXhQylD8Xy7wFLVHVv


## Fine-tuning

Now we can create our fine-tuning job with the generated files and an optional suffix to identify the model. The response will contain an `id` which you can use to retrieve updates on the job.

Note: The files have to first be processed by our system, so you might get a `File not ready` error. In that case, simply retry a few minutes later.


In [ ]:
response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-3.5-turbo",
    suffix="airline-ner",
)

job_id = response.id

print("Job ID:", response.id)
print("Status:", response.status)

Job ID: ftjob-RTzcyZn0tjXHNlRkkroB1Qxk
Status: validating_files


#### Check job status


In [ ]:
response = client.fine_tuning.jobs.retrieve(job_id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)


Job ID: ftjob-RTzcyZn0tjXHNlRkkroB1Qxk
Status: validating_files
Trained Tokens: None


We can track the progress of the fine-tune with the events endpoint.


In [ ]:
response = client.fine_tuning.jobs.list_events(job_id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 287/303: training loss=0.80
Step 288/303: training loss=0.95
Step 289/303: training loss=0.91
Step 290/303: training loss=0.79, validation loss=0.79
Step 291/303: training loss=0.75
Step 292/303: training loss=1.02
Step 293/303: training loss=1.25
Step 294/303: training loss=0.70
Step 295/303: training loss=0.58
Step 296/303: training loss=0.74
Step 297/303: training loss=0.78
Step 298/303: training loss=0.85
Step 299/303: training loss=0.87
Step 300/303: training loss=0.62, validation loss=0.71
Step 301/303: training loss=0.83
Step 302/303: training loss=0.82
Step 303/303: training loss=0.83, full validation loss=0.86
Checkpoint created at step 101 with Snapshot ID: ft:gpt-3.5-turbo-0125:personal:airline-ner:9NvkGYNV:ckpt-step-101
Checkpoint created at step 202 with Snapshot ID: ft:gpt-3.5-turbo-0125:personal:airline-ner:9NvkGRl0:ckpt-step-202
New fine-tuned model created: ft:gpt-3.5-turbo-0125:personal:airline-ner:9NvkGplz


Fine-tuned model ID from the job:

In [ ]:
response = client.fine_tuning.jobs.retrieve(job_id)
fine_tuned_model_id = response.fine_tuned_model

if fine_tuned_model_id is None:
    raise RuntimeError("Fine-tuned model ID not found. Your job has likely not been completed yet.")

print("Fine-tuned model ID:", fine_tuned_model_id)

Fine-tuned model ID: ft:gpt-3.5-turbo-0125:personal:airline-ner:9NvkGplz


## Inference

In [ ]:
test_df = airline_df.loc[201:300]
test_row = test_df.iloc[0]
test_messages = []
test_messages.append({"role": "system", "content": system_message})
user_message = create_user_message(test_row)
test_messages.append({"role": "user", "content": user_message})

pprint(test_messages)

[{'content': 'You are a helpful travel assistant. Given the Start Date, '
             'Departure City, Destination City, you will provide the shortest '
             'travel details, including departure airport, destination '
             'airport, airline, flight number, layover details (if any), and '
             'total travel duration.',
  'role': 'system'},
 {'content': 'Start Date: 06/21/2023\n'
             'Departure City: Everett\n'
             'Destination City: Nova Iguacu\n'
             '\n'
             'Based on the given information, please provide the most suitable '
             'travel plan.',
  'role': 'user'}]


In [ ]:
response = client.chat.completions.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=500
)
print(response.choices[0].message.content)

Snohomish County (Paine Field)
Everett
United States
Aeroporto de Nova Iguacu
Nova Iguacu
Brazil
Hernandez, Johnson and Johnson
JQK292
Kampong Chhnang Airport
Kompong Chhnang
Cambodia
6.0
11.0


In [ ]:
import openai

# Set your OpenAI API key
openai.api_key = "api_key"

def set_open_params(model=None, temperature=0.7, max_tokens=256, top_p=1, frequency_penalty=0, presence_penalty=0):
    """Set OpenAI parameters"""
    openai_params = {}
    if model is None:
        print("Please provide a valid fine-tuned model ID.")
        return None
    openai_params['model'] = model
    openai_params['temperature'] = temperature
    openai_params['max_tokens'] = max_tokens
    openai_params['top_p'] = top_p
    openai_params['frequency_penalty'] = frequency_penalty
    openai_params['presence_penalty'] = presence_penalty
    return openai_params

def get_completion(params, messages):
    """GET completion from OpenAI API"""
    response = openai.chat.completions.create(
        model=params['model'],
        messages=messages,
        temperature=params['temperature'],
        max_tokens=params['max_tokens'],
        top_p=params['top_p'],
        frequency_penalty=params['frequency_penalty'],
        presence_penalty=params['presence_penalty']
    )
    return response.choices[0].message

# Test with your fine-tuned model
fine_tuned_model_id = "ft:gpt-3.5-turbo-0125:personal:airline-ner:9NvkGplz"
params = set_open_params(model=fine_tuned_model_id)

if params is not None:
    prompt = "07/15/2023,Guarapuava,Abu Dhabi\n\nBased on the given information, please provide the shortest travel details."
    messages = [{"role": "system", "content": "You are a helpful travel assistant."}, {"role": "user", "content": prompt}]
    output = get_completion(params, messages)
    print("Prompt:", prompt)
    print("Output:", output)

Prompt: 07/15/2023,Guarapuava,Abu Dhabi

Based on the given information, please provide the shortest travel details.
Output: ChatCompletionMessage(content='Guarapuava, Abu Dhabi, Brazil\nGuarapuava\nBrazil', role='assistant', function_call=None, tool_calls=None)


In [ ]:
if params is not None:
    prompt = "10/15/2023,Beverly,Dirkou\n\nBased on the given information, please provide the shortest travel details."
    messages = [{"role": "system", "content": "You are a helpful travel assistant."}, {"role": "user", "content": prompt}]
    output = get_completion(params, messages)
    print("Prompt:", prompt)
    print("Output:", output)

Prompt: 10/15/2023,Beverly,Dirkou

Based on the given information, please provide the shortest travel details.
Output: ChatCompletionMessage(content='Beverly\nNigeria\nDirkou\nNiger\n3,099.0\n3.25', role='assistant', function_call=None, tool_calls=None)


In [ ]:
if params is not None:
    prompt = "08/20/2023,Quilpie,Tepic\n\nBased on the given information, please provide the shortest travel details."
    messages = [{"role": "system", "content": "You are a helpful travel assistant."}, {"role": "user", "content": prompt}]
    output = get_completion(params, messages)
    print("Prompt:", prompt)
    print("Output:", output)

Prompt: 08/20/2023,Quilpie,Tepic

Based on the given information, please provide the shortest travel details.
Output: ChatCompletionMessage(content='Quilpie, Queensland, Australia\nTepic, Nayarit, Mexico', role='assistant', function_call=None, tool_calls=None)


In [ ]:
if params is not None:
    prompt = "05/30/2023,Liege,Mercury\n\nBased on the given information, please provide the shortest travel details."
    messages = [{"role": "system", "content": "You are a helpful travel assistant."}, {"role": "user", "content": prompt}]
    output = get_completion(params, messages)
    print("Prompt:", prompt)
    print("Output:", output)

Prompt: 05/30/2023,Liege,Mercury

Based on the given information, please provide the shortest travel details.
Output: ChatCompletionMessage(content='Liege\nMercury\nBelgium\n-17.625\n29.5', role='assistant', function_call=None, tool_calls=None)
